In [1]:
from __future__ import print_function
import numpy as np 
np.random.seed(1337) 

In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Activation, Flatten, Input, MaxPooling2D
from keras.layers import Conv2D
import h5py  # to ensure we have this package installed 

from keras.callbacks import ModelCheckpoint

/opt/app-root/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/opt/app-root/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/app-root/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/app-root/lib/python3.6/site-packages/tensorflow/python/framework/dtyp

In [3]:
batch_size = 32
num_classes = 10
epochs = 10

In [4]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Inception Module

Take a look at the inception module figure here: https://arxiv.org/pdf/1409.4842.pdf

In [6]:
# input layer is the same as our typical CNN model 
inputs = Input(shape=(32, 32, 3))

## ----------- New Stuff Starts Here --------- 


tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(inputs)
tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu', name='t1_conv')(tower_1)

tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(inputs)
tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)

x = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1) 

tower_4 = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
tower_4 = Conv2D(64, (3, 3), padding='same', activation='relu', name='t4_conv')(tower_4)

tower_5 = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
tower_5 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_5)

tower_6 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
tower_6 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_6)

x = keras.layers.concatenate([tower_4, tower_5, tower_6], axis=1) 

## ----------- New Stuff Ends Here --------- 

# Rest of the model, again, remains the same 

x = Conv2D(8, (3, 3))(x)    
x = Activation('relu')(x) 
x = MaxPooling2D(pool_size=(2, 2))(x) 
x = Flatten()(x) 

x = Dense(num_classes)(x) 

output = Activation('softmax')(x) 



W0729 19:41:50.383115 140298476656448 deprecation_wrapper.py:119] From /opt/app-root/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0729 19:41:50.391854 140298476656448 deprecation_wrapper.py:119] From /opt/app-root/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0729 19:41:50.393748 140298476656448 deprecation_wrapper.py:119] From /opt/app-root/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0729 19:41:50.423109 140298476656448 deprecation_wrapper.py:119] From /opt/app-root/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [7]:
model = Model([inputs], output)

In [8]:
model.summary()  # Notice the 'Connected To' in the summary 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   256         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 64)   256         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 32, 32, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
t1_conv (C

In [9]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

filepath="../checkpoints/cifar10-inception2-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_acc', 
                             verbose=1, 
                             mode='max')

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          callbacks=[checkpoint])

W0729 19:41:50.543889 140298476656448 deprecation_wrapper.py:119] From /opt/app-root/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0729 19:41:50.549692 140298476656448 deprecation_wrapper.py:119] From /opt/app-root/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0729 19:41:50.604745 140298476656448 deprecation.py:323] From /opt/app-root/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0729 19:41:50.726263 140298476656448 deprecation_wrapper.py:119] From /opt/app-root/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add i

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
 4800/50000 [=>............................] - ETA: 2:26 - loss: 2.0944 - acc: 0.2365

KeyboardInterrupt: 

## Next Steps 

Now you are in the position to go ahead and explore the Keras documentation on your own. There are some great examples here https://keras.io/getting-started/functional-api-guide/#more-examples

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")